In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!folder=19_06 && cp drive/MyDrive/ITMO/samovar/validation/$folder ./ -r && mv ./$folder ./data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mv: cannot move './19_06' to './data/19_06': Directory not empty


In [ ]:
!wget https://ftp.ncbi.nih.gov/pub/taxonomy/new_taxdump/new_taxdump.tar.gz
!tar -xvzf new_taxdump.tar.gz
!mkdir ncbi && mv *dmp ncbi/
!rm *tar*

--2024-09-19 14:24:51--  https://ftp.ncbi.nih.gov/pub/taxonomy/new_taxdump/new_taxdump.tar.gz
Resolving ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)... 130.14.250.12, 130.14.250.10, 130.14.250.11, ...
Connecting to ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137907580 (132M) [application/x-gzip]
Saving to: ‘new_taxdump.tar.gz’

new_taxdump.tar.gz  100%[===================>] 131.52M   148MB/s    in 0.9s    

2024-09-19 14:24:52 (148 MB/s) - ‘new_taxdump.tar.gz’ saved [137907580/137907580]

citations.dmp
delnodes.dmp
division.dmp
excludedfromtype.dmp
fullnamelineage.dmp
gencode.dmp
host.dmp
images.dmp
merged.dmp
names.dmp
nodes.dmp
rankedlineage.dmp
taxidlineage.dmp
typematerial.dmp
typeoftype.dmp
mkdir: cannot create directory ‘ncbi’: File exists


## Import modules

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import requests
import re
!pip install taxonomy
import taxonomy

## Functions

In [ ]:
if False:
  def get_GMrepo_runs(mesh_id = 'D006262'):
    # code from https://github.com/evolgeniusteam/GMrepoProgrammableAccess/blob/master/programmable-access/
    pheno_query = {'mesh_id':mesh_id}
    url = 'http://gmrepo.humangut.info/api/countAssociatedRunsByPhenotypeMeshID'
    pheno = requests.post(url, data=json.dumps(pheno_query))
    pheno_cont = pheno.json()

    ## -- the resulting variable is a vector --
    phenotyp_nr_assoc_runs = pd.DataFrame(pheno.json())
    print('Number of associated runs:', len(phenotyp_nr_assoc_runs))
    return phenotyp_nr_assoc_runs
  get_GMrepo_runs().head()

In [ ]:
def read_kaiju_raw(file_path):
  df = pd.read_table(file_path, header = None)
  df.columns = ["classified", "seq", "score", "taxID", "N"]
  return(df)

read_kaiju_raw("data/hiseq_kaiju.log").head()

,classified,seq,score,taxID,N
0,C,Ecoli.fna-NC_000913.3_52_0,126.0,"CM000664.2,CM000672.2,CM000671.2,CM000663.2,CM...",NaN
1,C,Ecoli.fna-NC_000913.3_79_1,100.0,"GL383571.1,KI270442.1,KI270435.1,KI270736.1,KI...",NaN
2,C,Ecoli.fna-NC_000913.3_44_2,128.0,"CM000663.2,",NaN
3,C,Ecoli.fna-NC_000913.3_41_3,110.0,"KI270730.1,KI270741.1,CM000666.2,CM000680.2,CM...",NaN
4,C,Ecoli.fna-NC_000913.3_27_4,172.0,"KI270738.1,KI270739.1,KZ208915.1,KI270741.1,CM...",NaN


In [ ]:
def read_kraken1_raw(file_path):
  df = pd.read_table(file_path, header = None)
  df.columns = ["classified", "seq", "taxID", "length", "k-mer"]
  return(df)

read_kraken1_raw("data/hiseq_k1.log").head()


,classified,seq,taxID,length,k-mer
0,C,Scer.fna-NC_001224.1_5_4,9606,253,0:30 9606:1 0:65 A:31 0:96
1,U,Scer.fna-NC_001224.1_6_4,0,253,0:96 A:31 0:96
2,U,Scer.fna-NC_001224.1_7_4,0,253,0:96 A:31 0:96
3,U,Scer.fna-NC_001224.1_8_4,0,253,0:96 A:31 0:96
4,U,Scer.fna-NC_001224.1_9_4,0,253,0:96 A:31 0:96


In [ ]:
def read_kraken2_raw(file_path):
  df = pd.read_table(file_path, header = None)
  df.columns = ["classified", "seq", "taxa", "length", "k-mer"]
  df["length"] = [re.sub("\|.*", "", i)for i in df["length"] ]
  df["taxID"] = [re.search(r'(?<=taxid )[0-9]*',i).group(0) for i in df["taxa"]]
  df["taxa"] = [re.sub(r' \(.*', "", i) for i in df["taxa"]]
  return(df)

read_kraken2_raw("data/hiseq_k2.log").head()

,classified,seq,taxa,length,k-mer,taxID
0,C,Ecoli.fna-NC_000913.3_0_0,Escherichia coli str. K-12 substr. MG1655,126,511145:92 |:| 511145:92,511145
1,C,Ecoli.fna-NC_000913.3_1_0,Escherichia coli str. K-12 substr. MG1655,126,511145:92 |:| 511145:92,511145
2,C,Ecoli.fna-NC_000913.3_2_0,Escherichia coli str. K-12 substr. MG1655,126,511145:92 |:| 511145:92,511145
3,C,Ecoli.fna-NC_000913.3_3_0,Escherichia coli str. K-12 substr. MG1655,126,511145:92 |:| 511145:92,511145
4,C,Ecoli.fna-NC_000913.3_4_0,Escherichia coli str. K-12 substr. MG1655,126,511145:92 |:| 511145:92,511145


In [ ]:
def read_krakenu_raw(file_path):
  df = pd.read_table(file_path, header = None)
  df.columns = ["classified", "seq", "taxID", "length", "k-mer"]
  return(df)

read_krakenu_raw("data/hiseq_ku.log").head()

,classified,seq,taxID,length,k-mer
0,C,Scer.fna-NC_001224.1_5_4,9606,253,0:30 9606:1 0:65 A:31 0:96
1,U,Scer.fna-NC_001224.1_6_4,0,253,0:96 A:31 0:96
2,U,Scer.fna-NC_001224.1_7_4,0,253,0:96 A:31 0:96
3,U,Scer.fna-NC_001224.1_8_4,0,253,0:96 A:31 0:96
4,U,Scer.fna-NC_001224.1_9_4,0,253,0:96 A:31 0:96


In [ ]:
def read_mp4_raw(file_path):
  df = pd.read_table(file_path, header = None)
  df["seq"] = [re.sub("\/.*", "", i) for i in df[0]]
  return df

read_mp4_raw("data/hiseq_mpa.log").head()

,0,1,seq
0,Ecoli.fna-NC_000913.3_8_0/1__1.9,VDB|0046-01F9-0-0000|M367-c418-c0-c0,Ecoli.fna-NC_000913.3_8_0
1,Ecoli.fna-NC_000913.3_10_0/1__1.11,VDB|0035-0014-0-0009|M367-c467-c0-c0,Ecoli.fna-NC_000913.3_10_0
2,Ecoli.fna-NC_000913.3_11_0/1__1.12,VDB|0024-0000-0-0001|M1206-c595-c0-c0,Ecoli.fna-NC_000913.3_11_0
3,Ecoli.fna-NC_000913.3_13_0/1__1.14,VDB|0046-01F9-0-0000|M367-c418-c0-c0,Ecoli.fna-NC_000913.3_13_0
4,Ecoli.fna-NC_000913.3_45_0/1__1.46,VDB|004C-00C4-0-0003|M367-c125-c0-c0,Ecoli.fna-NC_000913.3_45_0


In [ ]:
read_raw = {
    "kraken": read_kraken1_raw,
    "kraken1": read_kraken1_raw,
    "kraken2": read_kraken2_raw,
    "krakenunique": read_krakenu_raw,
    "krakenu": read_krakenu_raw,
    "metaphaln": read_mp4_raw,
    "mpa": read_mp4_raw,
    "mp4": read_mp4_raw,
    "kaiju": read_kaiju_raw
}

In [ ]:
def read_annotation(file_path_type: dict, trimmed = True):
  #file_path_type: {path: type}
  res = pd.DataFrame()
  for a, b in file_path_type.items():
    df = read_raw.get(b)(a).set_index("seq")
    if trimmed:
      df = df[["taxID"]]

    df.columns = [i + "_" + b if i != "seq" else "seq" for i in df.columns]
    res = pd.concat([res, df], axis = 1)
  return res

read_annotation({
    "data/hiseq_k2.log": "kraken2",
    "data/hiseq_k1.log": "kraken1",
    "data/hiseq_ku.log": "krakenu"
}).head()

,taxID_kraken2,taxID_kraken1,taxID_krakenu
seq,,,
Ecoli.fna-NC_000913.3_0_0,511145,511145,511145
Ecoli.fna-NC_000913.3_1_0,511145,511145,511145
Ecoli.fna-NC_000913.3_2_0,511145,511145,511145
Ecoli.fna-NC_000913.3_3_0,511145,511145,511145
Ecoli.fna-NC_000913.3_4_0,511145,511145,511145


In [ ]:
tax = taxonomy.Taxonomy.from_ncbi("./ncbi")

In [ ]:
class annotation:
  #dictionary
  read_raw = {
    "kraken": read_kraken1_raw,
    "kraken1": read_kraken1_raw,
    "kraken2": read_kraken2_raw,
    "krakenunique": read_krakenu_raw,
    "krakenu": read_krakenu_raw,
    "metaphaln": read_mp4_raw,
    "mpa": read_mp4_raw,
    "mp4": read_mp4_raw,
    "kaiju": read_kaiju_raw
  }

  #init
  def __init__(self, file_path_type:dict, get_true_annotation = None):
    self.DataFrame = pd.DataFrame()
    for a, b in file_path_type.items():
      df = annotation.read_raw.get(b)(a).set_index("seq").astype({"taxID": 'string'})
      df.columns = [i + "_" + b for i in df.columns]
      self.DataFrame = pd.concat([self.DataFrame, df], axis = 1)

    if get_true_annotation is not None:
      self.true_annotation = [re.search(get_true_annotation, i).group(0) for i in self.DataFrame.index]

    set_columns = []
    for name, column in self.tr().items():
      set_columns += list(set(column))

    self.annotation_list = annotation.list2set(set_columns)
    self.true_annotation_list = annotation.list2set(self.true_annotation)
    self.rank = annotation.list2set([*self.annotation_list, *self.true_annotation_list])

  # true annotation
  def true_annotation_unique (self):
    return set(self.true_annotation)

  def true_annotation_rename(self, change_dict:dict):
    self.true_annotation = [change_dict.get(TA) if TA in change_dict.keys() else "" for TA in self.true_annotation]

  # expand annotation levels
  def rank_annotation(self, rank = "species"):
    rank_list = [annotation.rank(j, rank) for j in self.rank]
    rank_dict = dict(zip(self.rank, rank_list))
    return rank_annotation(rank).make(self.full(), rank_dict)

  def expand_annotation(self, rank = ["species"]):
    full_rank_annotation = expand_annotation()
    for i in rank:
      full_rank_annotation.add(self.rank_annotation(i))

    return full_rank_annotation

  def correct_annotations(self, rank = "species"):
    return pd.DataFrame(self.rank_annotation(rank).correct_annotation().annotation.value_counts())


  # forms
  def full(self):
    tmp = self.tr()
    tmp["true"] = self.true_annotation
    return tmp

  def tr(self):
    return self.DataFrame.copy().filter(regex="taxID.*")

  #misc
  def list2set(a:list):
    return list(set([str(i) for i in a]))

  def list2rank(a, at_rank):
    b = annotation.list2set(a)
    return [tax.node(i) for i in b]

  def rank(j,i):
    if j == "0":
      return "0"
    tmp = tax.parent(j, at_rank = i)
    if tmp is not None:
      return str(tmp.id)
    else:
      return None

  def export(self, file = False):
    df_return = self.DataFrame.loc[:, [col for col in self.DataFrame if col.startswith('taxID')]]
    df_return["length"] = self.DataFrame.loc[:, [col for col in self.DataFrame if col.startswith('length')][1] ]
    df_return["true"] = self.true_annotation
    if file != False:
      df_return.to_csv(file)
    return(df_return)



class rank_annotation:

  #prepare
  def __init__(self, rank):
    self.rank = rank
    self.annotation = pd.DataFrame()

  def add(self, name, annotation):
    self.annotation[name] = annotation

  def make(self, annotation: pd.DataFrame, rank_dict: dict):
    for name, column in annotation.items():
      self.add(str(name), [rank_dict.get(TA) for TA in column])
    self.reindex(annotation.index)
    return self

  def reindex(self, index):
    self.annotation.index = index

  #split and eval
  def y(self):
    return self.annotation["true"]

  def x(self):
    return self.annotation.copy().drop("true", axis = 1)

  def correct_annotation(self):
    tmp = rank_annotation(self.rank)

    for name, column in self.x().items():
      tmp.add(str(name), pd.DataFrame(column == self.y()))

    tmp.reindex(self.annotation.index)
    return tmp




class expand_annotation:

  def __init__(self):
    self.rank_annotation = {}

  def add(self, rank_annotation):
    self.rank_annotation[rank_annotation.rank] = rank_annotation

  def get(self, rank):
    return self.rank_annotation[rank]

## Import data

In [ ]:
df = annotation({
    "data/hiseq_k1.log": "kraken1",
    "data/hiseq_k2.log": "kraken2",
    "data/hiseq_ku.log": "krakenu"
}, get_true_annotation=".*(?=.fna)")

df.true_annotation_rename({
    'Ecoli': '511145',
    'Hsap': '9606',
    'Phix': '2886930',
    'Scer': '559292'
})

df.correct_annotations()

,,,count
taxID_kraken1,taxID_kraken2,taxID_krakenu,
True,True,True,1905
False,False,False,91
True,False,True,6


In [ ]:
df.export("toy_matrix.csv")

,taxID_kraken1,taxID_kraken2,taxID_krakenu,length,true
seq,,,,,
Scer.fna-NC_001224.1_5_4,9606,0,9606,126,559292
Scer.fna-NC_001224.1_6_4,0,0,0,126,559292
Scer.fna-NC_001224.1_7_4,0,0,0,126,559292
Scer.fna-NC_001224.1_8_4,0,0,0,126,559292
Scer.fna-NC_001224.1_9_4,0,0,0,126,559292
...,...,...,...,...,...
Scer.fna-NC_001224.1_0_4,0,0,0,126,559292
Scer.fna-NC_001224.1_1_4,559292,0,559292,126,559292
Scer.fna-NC_001224.1_2_4,0,0,0,126,559292


## Тесты

In [ ]:
# add https://github.com/rrwick/Metagenomics-Index-Correction